In [1]:
import requests
import bs4
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [ ]:
'Clusone', 'Borghi', 'Rota Greca', 'Dubino', 'Valfurva',

In [2]:
df = pd.read_csv('inf_comunali.csv')
df[df['Comune'] == 'Cortale']

,Municipio,Provincia,Distanza dal capoluogo,Abitanti,Denominazione,Densità abitativa,Comune,Regione,Zona,sigla_provincia,Età (Media),Parrocchie,Scuole,Industria Addetti,Servizi Addetti,Amministrazione Addetti,Altro Addetti,Diplomati,Laureati,Liscenziati Midia Totale,Licenza Elementare Totale,Alfabeti,Analfabeti,Media annuale (Redditi),Media Mensile (Redditi),Imposta netta annuale,Imposta netta mensile,Totale di stranieri
4163,Via F. Turati 18,Catanzaro,41 Km,2.117,Cortalesi,"83,14",Cortale,Calabria,Sud,CZ,46.57,2,3,7.8%,8.4%,6.6%,10.2%,525,219,486,449,172,79,11814.75,984.56,2988.95,249.08,36


In [3]:
df.Comune.value_counts()

Comune
Agrigento                     1
Grottole                      1
San Giorgio delle Pertiche    1
Poggiodomo                    1
Castello di Cisterna          1
                             ..
Anzano di Puglia              1
Lozza                         1
San Lorenzo Nuovo             1
Capriva del Friuli            1
Vittorio Veneto               1
Name: count, Length: 5680, dtype: int64

In [7]:
df = pd.read_csv('inf_parrocchie.csv')
df[df['Comune'] == 'Monza']

,Chiesa,Comune,Frazione,Indirizzo,Parrrocchiani,Diocesi,Titolo:,Unnamed: 7,Nome,Cap,Tipo,P.IVA,Street View,Unnamed: 13,Regione,Provincia,Banca,ABI,CAB,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22


In [25]:
# URL della pagina
#url_comune = 'https://italia.indettaglio.it/ita/calabria/civita.html'
url_comune = 'https://italia.indettaglio.it/ita/emiliaromagna/solarolo.html'

# Imposta il comune
comune = "Solarolo"

# Eseguire la richiesta HTTP
requisicao = requests.get(url_comune, verify=False)
pagina_del_comune = bs4.BeautifulSoup(requisicao.text, "html.parser")

df_dic = {}

c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [26]:
def pulisci_dataframe(data_dict, colonne_da_rinominare):
    """
    Pulisce un DataFrame disordinato, aggiungendo automaticamente la prima riga 
    e rinominando le colonne specificate.

    :param data_dict: Dizionario dei dati grezzi
    :param colonne_da_rinominare: Lista di nomi per le colonne finali
    :return: DataFrame pulito
    """
    
    # Convertiamo il dizionario in un DataFrame temporaneo
    df_temp = pd.DataFrame(data_dict)

    # Recuperiamo automaticamente la prima riga (nomi effettivi delle colonne)
    prima_riga = [col.replace(":", "").strip() for col in df_temp.columns[:len(colonne_da_rinominare)]]
    
    # Creiamo un nuovo DataFrame con la prima riga inclusa
    df_clean = pd.DataFrame({
        colonne_da_rinominare[i]: [prima_riga[i]] + df_temp.iloc[:, i].str.replace(":", "").str.strip().tolist()
        for i in range(len(colonne_da_rinominare))
    })

    # Pulizia e conversione dei numeri
    for col in df_clean.columns:
        if col == "Percentuale_Totale": #.str.contains("%").any():  # Se contiene percentuali
            df_clean[col] = df_clean[col].str.replace(",", ".").str.rstrip("%").astype(float) / 10
        elif df_clean[col].str.replace(".", "", regex=False).str.isnumeric().all():  # Se è un numero
            df_clean[col] = df_clean[col].str.replace(".", "", regex=False).astype(int)  # Rimuove i punti

    return df_clean

def transform_dict_to_df(data_dict):
    df = pd.DataFrame(data_dict)
    new_data = {}

    for col in df.columns:
        if col != list(data_dict.keys())[0]:  
            for idx, age_group in df[list(data_dict.keys())[0]].items():
                new_col_name = f"{col}_{age_group}"
                new_data[new_col_name] = df.at[idx, col]  # Aggiungiamo il valore corrispondente

    # Creiamo il nuovo DataFrame con una sola riga
    new_df = pd.DataFrame([new_data])
    
    return new_df

def concat_dataframe(accesso, df2):
    df1 = pd.read_csv(accesso)
    df = pd.concat([df1, df2], ignore_index=True)
    df.to_csv(accesso, index=False)

    return df


In [27]:
# Trova il paragrafo con le informazioni
dati_paragrafo = pagina_del_comune.find("p", class_="well")

if dati_paragrafo:
    testo = dati_paragrafo.get_text()

    # Estrai le informazioni con slicing testuale
    info = {}
    info["Municipio"] = testo.split("Municipio: ")[1].split(",")[0]
    info["Provincia"] = testo.split("Provincia: ")[1].split(".")[0]
    info["Distanza dal capoluogo"] = testo.split("Distanza dal capoluogo")[1].split(": ")[1].split(" ")[0] + " Km"
    info["Abitanti"] = testo.split("Abitanti: ")[1].split(" ")[0]
    info["Denominazione"] = testo.split("Denominazione: ")[1].split(".")[0]

    # Stampare il risultato
    for k, v in info.items():
        df_dic[k] = v
        print(f"{k}: {v}")

else:
    print("Errore: Impossibile trovare il paragrafo con i dati richiesti.")

Municipio: Piazza Gonzaga 1
Provincia: Ravenna
Distanza dal capoluogo: 34 Km
Abitanti: 4.488
Denominazione: Solarolesi


In [28]:
# Trova il paragrafo che contiene la densità abitativa
densita_text = None
for p in pagina_del_comune.find_all("p"):
    if "Densità abitativa" in p.text:
        densita_text = p.text
        break

# Estrai il numero dalla stringa trovata
if densita_text:
    match = re.search(r"Densità abitativa:\s*([0-9.,]+)", densita_text)
    if match:
        densita = match.group(1)
        print(f"Densità abitativa: {densita} abitanti per km²")
        df_dic['Densità abitativa'] = densita
    else:
        print("Densità abitativa non trovata.")
else:
    print("Sezione sulla densità abitativa non trovata.")

Densità abitativa: 160,92 abitanti per km²


In [29]:
df_comune = pd.DataFrame([df_dic])
df_comune

,Municipio,Provincia,Distanza dal capoluogo,Abitanti,Denominazione,Densità abitativa
0,Piazza Gonzaga 1,Ravenna,34 Km,4.488,Solarolesi,"160,92"


In [30]:
# Cerca tutte le <div> che contengono tabelle (le tabelle sono racchiuse in div con classe "tb_resp")
div_tables = pagina_del_comune.find_all("div", class_="tb_resp")

families_table = None
for div in div_tables:
    table = div.find("table")
    if table:
        # Verifica se la prima riga (con classe "info") contiene "Numero di componenti"
        header_row = table.find("tr", class_="info")
        if header_row:
            cells = header_row.find_all("td")
            if cells and "Numero di componenti" in cells[0].get_text():
                families_table = table
                break

if families_table:
    # Estrazione delle righe della tabella
    rows = []
    for tr in families_table.find_all("tr"):
        cols = [td.get_text(strip=True) for td in tr.find_all("td")]
        rows.append(cols)
    
    # La struttura della tabella è la seguente:
    # Prima riga (header): ["Numero di componenti", "1", "2", "3", "4", "5", "6 o più"]
    # Seconda riga (dati): ["Numero di famiglie", "391", "390", "244", "156", "50", "32"]
    header = rows[0]
    data = rows[1]
    
    # Creare il DataFrame con la prima riga come intestazione
    df = pd.DataFrame([data], columns=header)
    df.rename(columns={"6 o piÃ¹": "6 o più"}, inplace=True)
    
    #print("Tabella 'Famiglie e loro numerosità di componenti':")
    #print(df)
    
    # Salva il risultato in un file CSV
    #df.to_csv("famiglie_solarolo.csv", index=False)
else:
    print("Tabella 'Famiglie e loro numerosità di componenti' non trovata.")

famiglie = df.drop(columns=["Numero di componenti"])
famiglie["Comune"] = comune
famiglie#.to_csv("inf_famiglie.csv", index=False)

,1,2,3,4,5,6 o più,Comune
0,391,390,244,156,50,32,Solarolo


In [31]:
def table(title):
    # Trova il titolo della sezione desiderata
    titolo_sezione = pagina_del_comune.find("span", id="accatre", string=title)
    # Dati sulla popolazione straniera residente a Solarolo

    if titolo_sezione:
        # Trova la tabella che segue il titolo della sezione
        tabella = titolo_sezione.find_next("table", class_="table")

        if tabella:
            # Estrai le intestazioni
            intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

            # Estrai i dati della tabella
            dati = []
            for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                dati.append(colonne)

            # Creazione del DataFrame
            df = pd.DataFrame(dati, columns=intestazioni)
            
            # Salva il DataFrame in un file CSV
            #df.to_csv("popolazione_straniera_solarolo.csv", index=False, encoding="utf-8")

            #print("File CSV creato con successo: popolazione_straniera_solarolo.csv")
        else:
            print("Errore: Impossibile trovare la tabella della popolazione straniera.")
    else:
        print("Errore: Impossibile trovare la sezione 'Dati sulla popolazione straniera residente a Solarolo'.")
    
    return df

In [32]:
colonne_finali = ["Settore", "Numero_Aziende", "Addetti", "Percentuale_Totale"]
settori = pulisci_dataframe(table("Geografia, Anagrafe e Statistica").to_dict(), colonne_finali)
settori["Comune"] = comune
settori#.to_csv("inf_settori.csv", index=False)

,Settore,Numero_Aziende,Addetti,Percentuale_Totale,Comune
0,Industrie,61,Addetti,29.7,Solarolo
1,Servizi,74,Addetti,16.8,Solarolo
2,Amministrazione,25,Addetti,11.7,Solarolo
3,Altro,129,Addetti,42.4,Solarolo


In [33]:
eta = transform_dict_to_df(table(f"Dati sulla popolazione residente a {comune}")
                     .rename(columns={"EtÃ": "Età"})
                     .to_dict())
eta["Comune"] = comune
eta#.to_csv("inf_età.csv", index=False)

,0-4_Maschi,0-4_Femmine,0-4_Totale,5-9_Maschi,5-9_Femmine,5-9_Totale,10-14_Maschi,10-14_Femmine,10-14_Totale,15-19_Maschi,15-19_Femmine,15-19_Totale,20-24_Maschi,20-24_Femmine,20-24_Totale,25-29_Maschi,25-29_Femmine,25-29_Totale,30-34_Maschi,30-34_Femmine,30-34_Totale,35-39_Maschi,35-39_Femmine,35-39_Totale,40-44_Maschi,40-44_Femmine,40-44_Totale,45-49_Maschi,45-49_Femmine,45-49_Totale,50-54_Maschi,50-54_Femmine,50-54_Totale,55-59_Maschi,55-59_Femmine,55-59_Totale,60-64_Maschi,60-64_Femmine,60-64_Totale,65-69_Maschi,65-69_Femmine,65-69_Totale,70-74_Maschi,70-74_Femmine,70-74_Totale,>74_Maschi,>74_Femmine,>74_Totale,Comune
0,87,75,162,84,57,141,68,71,139,59,66,125,64,70,134,84,66,150,99,95,194,100,112,212,113,107,220,123,127,250,99,91,190,86,91,177,96,78,174,72,65,137,77,95,172,166,302,468,Solarolo


In [34]:
stranieri = transform_dict_to_df(table(f"Dati sulla popolazione straniera residente a {comune}")
                     .rename(columns={"EtÃ": "Età"})
                     .to_dict())

for x in list(stranieri.columns):
    nuovo = str(x).replace("PiÃ¹", "Più")
    stranieri.rename(columns={x: nuovo}, inplace=True)  # Modifica in-place

stranieri["Comune"] = comune
stranieri#.to_csv("inf_stranieri.csv", index=False)

,Maschi_Da 0 a 29 anni,Maschi_Da 30 a 54 anni,Maschi_Più di 54 anni,Femmine_Da 0 a 29 anni,Femmine_Da 30 a 54 anni,Femmine_Più di 54 anni,Totale_Da 0 a 29 anni,Totale_Da 30 a 54 anni,Totale_Più di 54 anni,Comune
0,96,89,13,102,86,19,198,175,32,Solarolo


In [35]:
scolarizzazione = transform_dict_to_df(table(f"Livelli di scolarizzazione a {comune}")
                     .to_dict())
scolarizzazione["Comune"] = comune
scolarizzazione#.to_csv("inf_scolarizzazione.csv", index=False)

,Laurea_Maschi,Laurea_Femmine,Laurea_Totale,Diploma_Maschi,Diploma_Femmine,Diploma_Totale,Licenza Media_Maschi,Licenza Media_Femmine,Licenza Media_Totale,Licenza Elementare_Maschi,Licenza Elementare_Femmine,Licenza Elementare_Totale,Alfabeti_Maschi,Alfabeti_Femmine,Alfabeti_Totale,Analfabeti_Maschi,Analfabeti_Femmine,Analfabeti_Totale,Comune
0,103,123,226,437,430,867,419,349,768,281,391,672,130,170,300,8,18,26,Solarolo


In [36]:
def correzione_moneta(valore):
    valore = str(valore).replace("â¬", "€")
    return valore

def table(title):
    # Trova il titolo della sezione desiderata
    titolo_sezione = pagina_del_comune.find("h4", string=title)

    if titolo_sezione:
        # Trova la tabella che segue il titolo della sezione
        tabella = titolo_sezione.find_next("table", class_="table")

        if tabella:
            # Estrai le intestazioni
            intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

            # Estrai i dati della tabella
            dati = []
            for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                dati.append(colonne)

            # Creazione del DataFrame
            df = pd.DataFrame(dati, columns=intestazioni)
            df["Comune"] = comune

            if "Reddito" in df.columns:
                df["Reddito"] = df["Reddito"].apply(correzione_moneta)
                df["Media annuale"] = df["Media annuale"].apply(correzione_moneta)
                df["Media mensile"] = df["Media mensile"].apply(correzione_moneta)
                df["Anno precedente"] = df["Anno precedente"].apply(correzione_moneta)
                df["Variazione"] = df["Variazione"].apply(correzione_moneta)

            # Salva il DataFrame in un file CSV
            #df.to_csv("edifici_solarolo_costruzione.csv", index=False, encoding="utf-8")

            #print("File CSV creato con successo: edifici_solarolo_costruzione.csv")
        else:
            print("Errore: Impossibile trovare la tabella degli edifici per data di costruzione.")
    else:
        print("Errore: Impossibile trovare la sezione 'Gli edifici a Solarolo per data di costruzione'.")
    
    return df

In [37]:
cronologia_edifici = table(f"Gli edifici a {comune} per data di costruzione").drop(columns=["Date"])
cronologia_edifici["Comune"] = comune
cronologia_edifici#.to_csv("inf_cronologia_edifici.csv", index=False)

,Prima del 1919,1919-45,1946-60,1961-70,1971-80,1981-90,1991-2000,2001-05,Dopo il 2005,Comune
0,39,34,184,82,94,57,65,43,30,Solarolo


In [38]:
numero_di_piani = table(f"Gli edifici a {comune} per numero di piani").drop(columns=["Numero di piani"])
numero_di_piani.rename(columns={"Quattro o piÃ¹": "Quattro o più"}, inplace=True)
numero_di_piani["Comune"] = comune
numero_di_piani#.to_csv("inf_numero_di_piani.csv", index=False)

,Uno,Due,Tre,Quattro o più,Comune
0,16,396,205,11,Solarolo


In [39]:
interni = table(f"Gli edifici a {comune} per numero di interni").drop(columns=["Numero di interni"])
interni["Comune"] = comune
interni#.to_csv("inf_interni.csv", index=False)

,Uno,Due,Da tre a quattro,Da cinque a otto,Da nove a quindici,Sedici e oltre,Comune
0,245,204,137,33,8,1,Solarolo


In [40]:
redditi_01 = table("Redditi e contribuenti per tipo di reddito")#.to_csv("inf_redditi_01.csv", index=False)
redditi_02 = table("Redditi, imposte e addizionali comunali e regionali")#.to_csv("inf_redditi_02.csv", index=False)
redditi_03 = table("Redditi e contribuenti per fasce di reddito")#.to_csv("inf_redditi_03.csv", index=False)

redditi_03

,Categoria,Contribuenti,Reddito,Media annuale,Media mensile,Anno precedente,Variazione,Comune
0,Reddito complessivo minore di zero euro,9,€ -46.027,"€ -5.114,11","€ -426,18","€ -520,01","€ 93,83",Solarolo
1,Reddito complessivo da 0 a 10.000 euro,919,€ 4.572.388,"€ 4.975,39","€ 414,62","€ 400,27","€ 14,35",Solarolo
2,Reddito complessivo da 10.000 a 15.000 euro,608,€ 7.587.028,"€ 12.478,66","€ 1.039,89","€ 1.046,76","€ -6,87",Solarolo
3,Reddito complessivo da 15.000 a 26.000,1.201,€ 23.676.757,"€ 19.714,20","€ 1.642,85","€ 1.647,86","€ -5,01",Solarolo
4,Reddito complessivo da 26.000 a 55.000 euro,604,€ 20.509.030,"€ 33.955,35","€ 2.829,61","€ 2.819,10","€ 10,51",Solarolo
5,Reddito complessivo da 55.000 a 75.000 euro,40,€ 2.456.773,"€ 61.419,33","€ 5.118,28","€ 5.114,28","€ 3,99",Solarolo
6,Reddito complessivo da 75.000 a 120.000 euro,39,€ 3.443.940,"€ 88.306,15","€ 7.358,85","€ 7.455,47","€ -96,62",Solarolo
7,Reddito complessivo oltre 120.000 euro,17,€ 3.127.254,"€ 183.956,12","€ 15.329,68","€ 14.477,63","€ 852,05",Solarolo


In [41]:
# Trova il titolo della sezione con una ricerca flessibile
titolo_sezione = None
for tag in pagina_del_comune.find_all(["h2", "h3", "span", "p"]):  # Controlliamo più tag possibili
    if re.search(r"Parrocchie", tag.get_text(), re.IGNORECASE):
        titolo_sezione = tag
        break

if titolo_sezione:
    # Trova la tabella che segue il titolo della sezione
    tabella = titolo_sezione.find_next("table")

    if tabella:
        # Estrai le intestazioni della tabella
        intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

        # Estrai i dati
        dati = []
        for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
            colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
            dati.append(colonne)

        # Creazione del DataFrame
        parrocchie = pd.DataFrame(dati, columns=intestazioni)

        # Salva il DataFrame in un file CSV
        #df.to_csv("parrocchie_solarolo.csv", index=False, encoding="utf-8")

        #print("✅ File CSV creato con successo: parrocchie_solarolo.csv")
    else:
        print("❌ Errore: Impossibile trovare la tabella delle parrocchie.")
else:
    print("❌ Errore: Non è stata trovata la sezione 'Parrocchie'.")

parrocchie#.to_csv("inf_parrocchie.csv", index=False)

,Chiesa,Comune,Frazione,Indirizzo,Parrrocchiani,Diocesi
0,S. Giovanni Battista in Castelnuovo,Solarolo,,"Via Fiume Santerno, 1",240,Diocesi di Imola
1,S. Maria Assunta,Solarolo,,"Via Beltrani, 4",3.000,Diocesi di Faenza - Modigliana
2,S. Maria in Casanola,Solarolo,,"Via Gaiano-Casanola, 10",333,Diocesi di Faenza - Modigliana
3,S. Maria in Felisio,Solarolo,,"Via Felisio, 6",290,Diocesi di Faenza - Modigliana
4,S. Mauro Abate in San Mauro,Solarolo,,"Via San Mauro, 16",455,Diocesi di Imola
5,S. Michele in Gaiano,Solarolo,,"Via Gaiano - Casanola, 80",292,Diocesi di Faenza - Modigliana


In [42]:
# Trova la sezione delle scuole
scuole_section = pagina_del_comune.find("a", {"name": "scuole"})

if scuole_section:
    table = scuole_section.find_next("table")  # Trova la tabella successiva

    # Estrarre tutte le righe della tabella
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.text.strip() for td in tr.find_all(["th", "td"])]
        rows.append(cells)

    # Se la tabella ha almeno una riga, usa la prima come intestazione
    if len(rows) > 1:
        headers = rows[0]  # Prima riga come intestazione
        data = rows[1:]     # Dati senza la prima riga
    else:
        headers = None
        data = rows

    # Creare un DataFrame Pandas
    scuole = pd.DataFrame(data, columns=headers if headers else None)

    # Stampare il risultato
    #print(df)
else:
    print("Tabella delle scuole non trovata.")

scuole#.to_csv("inf_scuole.csv", index=False)

,Nome,Tipo,Comune,Indirizzo,Email e Pec,Telefono e fax,CAP
0,S. Maria Assunta - Giuseppina Beltrani Cuccoli -,Scuola dell'Infanzia,Solarolo,Borgo Bennoli 12,smassunta@alice.it,054652153054652153,48027
1,Via Resistenza - Solarolo,Scuola dell'Infanzia,Solarolo,Via Resistenza 2,raic81800r@istruzione.it,054651185,48027
2,Pezzani Renzo,Scuola Primaria,Solarolo,Via G. Matteotti 3,raic81800r@istruzione.it,054651133054651133,48027
3,Ungaretti,Scuola Secondaria di Primo Grado,Solarolo,Via Kennedy 10,raic81800r@istruzione.it,0546511040546618063,48027


In [43]:
informazioni_geografiche = pd.read_csv("informazioni_geografiche.csv")
informazioni_geografiche.head()

,sigla_provincia,Comune,Provincia,Regione,Provincie nella regione (N°),Comuni nella regione (N°),Superficie (kmq) della Regione,Macro Zona della Regione,Zona Precisa della Regione,Raccolto
0,TO,Agliè,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest,0
1,TO,Airasca,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest,0
2,TO,Ala di Stura,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest,0
3,TO,Albiano d'Ivrea,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest,0
4,TO,Almese,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest,0


In [44]:
data = eta.to_dict()

# Intervalli di età e relativi punti medi
age_intervals = {
    '0-4': 2, '5-9': 7, '10-14': 12, '15-19': 17, '20-24': 22, '25-29': 27,
    '30-34': 32, '35-39': 37, '40-44': 42, '45-49': 47, '50-54': 52, '55-59': 57,
    '60-64': 62, '65-69': 67, '70-74': 72, '>74': 77  # Il gruppo ">74" usa 77 come stima
}

# Calcolare la somma ponderata per la popolazione totale
total_population = 0
weighted_sum = 0

# Calcolare somma ponderata e totale della popolazione
for key, value in data.items():
    if 'Totale' in key:  # Consideriamo solo i totali
        age_group = key.split('_')[0]  # Estrai il gruppo di età (es. '0-4')
        middle_age = age_intervals[age_group]  # Età media stimata per il gruppo
        total_population += int(value[0])  # Converti in intero
        weighted_sum += middle_age * int(value[0])  # Aggiungi il contributo ponderato

# Calcolare la media di età
average_age = round(weighted_sum / total_population, 2)
average_age

44.5

In [45]:
# Aggiungi il nome del comune
df_comune["Comune"] = comune

# Estrai i dati direttamente dalle colonne specifiche
df_comune["Regione"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["Regione"].iloc[0]
df_comune["Zona"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["Macro Zona della Regione"].iloc[0]
df_comune["sigla_provincia"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["sigla_provincia"].iloc[0]
df_comune["Età (Media)"] = average_age

# Aggiungi il numero di parrocchie e scuole
df_comune["Parrocchie"] = parrocchie.shape[0]
df_comune["Scuole"] = scuole.shape[0]

# Estrai le percentuali di addetti dai settori
df_comune["Industria Addetti"] = f'{round(settori[settori["Settore"] == "Industrie"]["Percentuale_Totale"].iloc[0], 2)}%'
df_comune["Servizi Addetti"] = f'{round(settori[settori["Settore"] == "Servizi"]["Percentuale_Totale"].iloc[0], 2)}%'
df_comune["Amministrazione Addetti"] = f'{round(settori[settori["Settore"] == "Amministrazione"]["Percentuale_Totale"].iloc[0], 2)}%'
df_comune["Altro Addetti"] = f'{round(settori[settori["Settore"] == "Altro"]["Percentuale_Totale"].iloc[0], 2)}%'

# Estrai i dati sulla scolarizzazione
df_comune["Diplomati"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Diploma_Totale"].iloc[0]
df_comune["Laureati"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Laurea_Totale"].iloc[0]
df_comune["Liscenziati Midia Totale"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Licenza Media_Totale"].iloc[0]
df_comune["Licenza Elementare Totale"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Licenza Elementare_Totale"].iloc[0]
df_comune["Alfabeti"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Alfabeti_Totale"].iloc[0]
df_comune["Analfabeti"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Analfabeti_Totale"].iloc[0]

# Redditi
df_comune["Media annuale (Redditi)"] = round(redditi_01['Media annuale'].str.replace("€", "").str.replace(".", "")
                        .str.replace(",", ".").astype(float).mean(), 2)

df_comune["Media Mensile (Redditi)"] = round(redditi_01['Media mensile'].str.replace("€", "").str.replace(".", "")
                        .str.replace(",", ".").astype(float).mean(), 2)

df_comune["Imposta netta annuale"] = float(redditi_02.loc[redditi_02["Categoria"] == "Imposta netta", "Media annuale"]
                                            .iloc[0].replace("€", "").replace(".", "").replace(",", ".").strip())

df_comune["Imposta netta mensile"] = float(redditi_02.loc[redditi_02["Categoria"] == "Imposta netta", "Media mensile"]
                                            .iloc[0].replace("€", "").replace(".", "").replace(",", ".").strip())

# Calcola il totale di stranieri
df_comune["Totale di stranieri"] = (int(stranieri[stranieri["Comune"] == comune]['Totale_Da 0 a 29 anni'].iloc[0]) 
                                    + int(stranieri[stranieri["Comune"] == comune]['Totale_Da 30 a 54 anni'].iloc[0])
                                    + int(stranieri[stranieri["Comune"] == comune]['Totale_Più di 54 anni'].iloc[0]))

df_comune#.to_csv('inf_comunali.csv', index=False)

,Municipio,Provincia,Distanza dal capoluogo,Abitanti,Denominazione,Densità abitativa,Comune,Regione,Zona,sigla_provincia,Età (Media),Parrocchie,Scuole,Industria Addetti,Servizi Addetti,Amministrazione Addetti,Altro Addetti,Diplomati,Laureati,Liscenziati Midia Totale,Licenza Elementare Totale,Alfabeti,Analfabeti,Media annuale (Redditi),Media Mensile (Redditi),Imposta netta annuale,Imposta netta mensile,Totale di stranieri
0,Piazza Gonzaga 1,Ravenna,34 Km,4.488,Solarolesi,"160,92",Solarolo,Emilia-Romagna,Nord,RA,44.5,6,4,29.7%,16.8%,11.7%,42.4%,867,226,768,672,300,26,19946.37,1662.2,4009.64,334.14,405


In [46]:
#concat_dataframe("informazioni_comunali.csv", df_comune)